### Setup Jetson Inferencing

### Setup Camera

In [1]:
from camera import Camera
from CV_FIND import CV_IMG_PROCESSOR

width = int(244)
height = int(244)
#camera = cv2.VideoCapture(0) # FOR USB WEBCAM
camera = Camera(width=width, height=height, capture_width=1280, capture_height=720, capture_fps=60, capture_flip=2)
main_cv = CV_IMG_PROCESSOR()

## Collect User Data
#### Initialize Training Module

In [2]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
from jetcam.utils import bgr8_to_jpeg
import numpy as np
import cv2
import ipywidgets

In [3]:
#from agent import DDPG
from DQN import DQNAgent
img_shape_1 = (80,80,1) # 40,60
action_size = 19
car_agent = DQNAgent(img_shape_1, action_size)

Using TensorFlow backend.
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 80, 80, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 32)        2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               590080    
_____________________________________________________

In [4]:
widget_test = ipywidgets.Image(format='jpeg', width=width, height=height/2)
display(widget_test)
def update_images():

    img = camera.value
    main_cv.getSteering(img,"canny")
    img2 = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    img2 = cv2.resize(img2, (80,80)) # 60,40
    main_cv.preprocessed_img = img2/255
    main_cv.raw_img = img
    main_cv.flag = True

#car_agent.noise.mu = 0 * np.ones(1)
#car_agent.noise.reset()

# CAM TESTING
for i in range(0,30):
    update_images()
    action = np.argmax(car_agent.model.predict(car_agent.conv_to_tensor(main_cv.preprocessed_img)))*10
    preview = np.squeeze(main_cv.preprocessed_img)*255  
    widget_test.value = bgr8_to_jpeg(preview)
    
# View convolution layer (experimental)
if(0):
    test = car_agent.get_conv([car_agent.conv_to_tensor(resized_img)]).reshape(32,9,14)
    print(test.shape)

    row = 8
    col = 4
    ix = 1
    for _ in range(row):
        for _ in range(col):
            # specify subplot and turn of axis
            ax = plt.subplot(row, col, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            plt.imshow(test[ix], cmap='gray')
            ix += 1
    plt.show()

Image(value=b'', format='jpeg', height='122.0', width='244')

In [5]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp((-9*x)+4))

def tan(x):
    return -np.tanh((4*x)-2)

def tight_tan(x):
    return -np.tanh((15*x)-3)

def tan2(x):
     return -np.tanh((3*x)-1)

In [6]:
import time
# For plotting while training
def episodic_plot(in_matrix, title="Loss"):
    fig2 = plt.figure()
    ax1 = plt.axes()
    fig2.set_size_inches(15,5)
    ax1.set_title(title)
    for i, p in enumerate(in_matrix):
        ax1.plot(p, label=i)
    ax1.legend()
    
BENCHMARK = True
SHOW_LIVE = True

if(BENCHMARK):
    loop_times = []
all_times = []    

### Agent Training

In [16]:
from approxeng.input.selectbinder import ControllerResource
from IPython.display import display, clear_output
import random as rn
import serial

# PI Cam 160 FOV 60 FPS @ 1280x720 (GSTREAMER SAYS 120?)
with serial.Serial('/dev/ttyUSB0', 1000000, timeout=1) as ser:
    with ControllerResource() as joystick:
        CONST_TIME = 0.017
        P = 4
        D = 0.03
        error_old = 0
        
        # Initialize
        TRAIN = False
        count = 1
        STEERING = 90
        THROTTLE = 90
        update_images()
        
        state = main_cv.preprocessed_img
        next_state = main_cv.preprocessed_img
        action = 0
        reward = 0
        
        DEBUG = 0
        mse_arr = []
        MSE = 0
        start_time = 0
        loop_start = 0
        option = 0b000
        AUTO_THROTTLE = False
        total_time = 0
        speed = 0
        
        while joystick.connected:
            if(BENCHMARK):
                start_time = time.time()
            
            if(joystick['cross'] is not None):  # SQUARE EXIT
                print("EXIT")
                output = "{:05d}-{:05d}\n".format(int(90), int(90))
                ser.write(bytes(output,'utf-8'))
                ser.close()
                break
                
            if(joystick['triangle'] is not None):  # TRIANGLE TOGGLE TRAIN
                if(DEBUG == False):
                    if(TRAIN):
                        all_times.append((time.time()-loop_start))
                        total_time = sum(all_times)
                        print("Paused", len(all_times))
                        option &= 0b110  # clear 3rd bit
                        TRAIN = False   
                    else:
                        option |= 0b001 # set 3rd bit
                        print("Start")
                        TRAIN = True
                        loop_start = time.time()
                    state = main_cv.preprocessed_img
                    time.sleep(0.2)
                    count = 1
                else:
                    print("Cannot train while debug")
                
                
            if(joystick['circle'] is not None):  # X DEBUG. WILL NOT MODIFY NETWORK
                if(DEBUG):
                    option &= 0b101 # clear 2nd
                    DEBUG = False   

                else:
                    option |= 0b010 # set 2nd
                    if(TRAIN):
                        all_times.append((time.time()-loop_start))
                        print("Timer paused while debug")
                        total_time = sum(all_times)
                        option &= 0b110  # clear 3rd bit train off
                        TRAIN = False
                    DEBUG = True
                state = main_cv.preprocessed_img
                time.sleep(0.2)
                count = 1
                
            if(joystick['square'] is not None):# LB for auto-throttle
                if(AUTO_THROTTLE):
                    option &= 0b011  # clear 1st
                    AUTO_THROTTLE = False
                else:
                    option |= 0b100  # set 1st
                    AUTO_THROTTLE = True
                time.sleep(0.2)
                
            # Update CV calc and images
            update_images()
                
            # PD Calculations
            error = (main_cv.angle-90) # -90 0 90 ERR
            P_T = error * P
            D_T = ((error-error_old)/CONST_TIME)*D
            cv_action = int(np.clip(((P_T-D_T)+90), 2, 178))
            error_old = error
            
            act_values = car_agent.model.predict(car_agent.conv_to_tensor(main_cv.preprocessed_img))
            agent_action = np.argmax(act_values[0])*10
            correct_action_value = act_values[0][int((cv_action/10))]
            predicted_value = act_values[0][np.argmax(act_values[0])]
            
            # Default
            THROTTLE = int((joystick['ly']+1)/2*180)
            #STEERING = int((joystick['rx']+1)/2*180)

            # Value used for training
            train_STEER = agent_action
            if(TRAIN):
                # EXPLORATION
                if(count%5 == 0):  # "Random" good sample
                    chance = rn.randint(0, 9)
                    if(chance == 9): # 
                        train_STEER = rn.randint(2, 178)
                    else: # 
                        train_STEER = cv_action
                        

                if(AUTO_THROTTLE):
                    if(STEERING < 30 or STEERING > 160): # to avoid getting stuck in tight corner
                        THROTTLE = 150
                        
            # OUTPUT CONTROL DEFAULT; Changes after model has stabilized
            STEERING = agent_action
            if(AUTO_THROTTLE):
                THROTTLE = int(-160) # RPM
                   
            # Some manual 
            if(joystick['l1'] is not None):# L trigger for manual
                STEERING = int((joystick['rx']+1)/2*180)
                THROTTLE = int((joystick['ly']+1)/2*180)

            '''
            # DEFAULT 
                TRAINING OFF
                USER IN CONTROL
                
            # TRAIN
                TRAINING MODEL
                CV IN CONTROL
                
            # DEBUG MODE (TRAIN ON)
                TRAINING OFF
                VIEW STATE UPDATES, MSE
                VIEW THE Q TABLE
            '''
            
            if(TRAIN or DEBUG):
                next_state = main_cv.preprocessed_img
                if(count % 5 == 0): # Frame Skipping
                    if(DEBUG == 0):
                        target = reward + car_agent.gamma * \
                           np.amax(car_agent.target_model.predict(car_agent.conv_to_tensor(next_state))[0])
                        target_f = car_agent.target_model.predict(car_agent.conv_to_tensor(state))
                        target_f[0][int(action/10)] = target
                        car_agent.model.fit(car_agent.conv_to_tensor(state), target_f, epochs=1, verbose=0)
                        car_agent.batch_id += 1
                        
                        # TAU UPDATE
                        if(count % 60 == 0):
                            car_agent.target_train()
                        
                    # SARSA at this moment (for STATE)
                    state = next_state.copy()
                    action = train_STEER
                    reward = tight_tan(abs(train_STEER-cv_action)/180) 
                count += 1                   
                
            # Performance indicator
            if(count % 30 == 0):
                if(len(mse_arr) < 30):
                    mse_arr.append((agent_action-cv_action)**2)
                    MSE = np.mean(mse_arr)
                else:
                    mse_arr.clear()
                
            if(SHOW_LIVE):
                output = "{:05d}-{:05d}\n".format(int(THROTTLE), int(STEERING))
                ser.write(bytes(output,'utf-8'))
                time.sleep(0.001)
                speed_in = (ser.readline())
                if speed_in:
                    speed = speed_in
                    
                info_str = "SPEED {:1d} Predict {:1d} CV {:1.3f} TIME {:1.2f}".format(int(speed), int(agent_action), cv_action, total_time)
                second_line = "MEM {:1d} ID {:1d} VAL {:1.4f} AGNT = {:1.4f}".format(len(car_agent.memory), car_agent.batch_id, correct_action_value, predicted_value)
                third_line = "Reward {:1.3f} ACTION {:1d} MSE {:1.3f}".format(reward, int(action/10), MSE)
                fourth_line = "State [Throttle,Debug,Train]: {0:03b}".format(option)
                
                if(DEBUG):
                    # Need to find another way..
                    clear_output(wait=True)
                    for i, j in enumerate(act_values[0]):
                        print("{:1d} {:02.5f}   ".format(i,j),end=" ")
                
                if(TRAIN):
                    preview = (cv2.resize(state, (244,244)).copy())*255
                else:
                    preview = main_cv.preview_img
                    #preview = cv2.cvtColor(main_cv.raw_img, cv2.COLOR_BGR2GRAY)
                    
                cv2.putText(preview, info_str, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0))
                cv2.putText(preview, second_line, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0))
                cv2.putText(preview, third_line, (0, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0))
                cv2.putText(preview, fourth_line, (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0))

                cv2.circle(preview, (int(main_cv.line[0]), int(122)), 4, (255,0,0),  -1)
                widget_train.value = bgr8_to_jpeg(preview)

            if(BENCHMARK):
                delta = (time.time() - start_time)
                loop_times.append(delta)
            
if(BENCHMARK):
    mean_delta = np.array(loop_times).mean()
    fps = 1 / mean_delta

    print('average(sec):{:.2f}, fps:{:.2f}, train_time {:2f}'.format(mean_delta, fps, total_time))

Start
Paused 5
Start
Paused 6
Start
Paused 7
Start
Paused 8
Start
Paused 9
Start
Paused 10
Start
Paused 11
Start
Paused 12
Start
Paused 13
Start
Paused 14
Start
Paused 15
Start
Paused 16
Start
Paused 17
Start
Paused 18
Start
Paused 19
Start
Paused 20
EXIT
average(sec):0.02, fps:41.60, train_time 2273.351768


In [10]:
main_cv.arc_length_min = 170
main_cv.canny_min = 130

In [11]:
# NEED TO RUN THIS CELL BEFORE ABOVE ONE
if(SHOW_LIVE):
    widget_train = ipywidgets.Image(format='jpeg', width=width, height=height/2)
    display(widget_train)

Image(value=b'', format='jpeg', height='122.0', width='244')

In [26]:
from approxeng.input.selectbinder import ControllerResource
from IPython.display import display, clear_output
import random as rn
import serial

widget_test = ipywidgets.Image(format='jpeg', width=width, height=height/2)
display(widget_test)
RECORD = True
Frames = []
speeds = []

# PI Cam 160 FOV 60 FPS @ 1280x720 (GSTREAMER SAYS 120?)
with serial.Serial('/dev/ttyUSB0', 1000000, timeout=1) as ser:
    with ControllerResource() as joystick:
        # Initialize
        STEERING = 90
        THROTTLE = 90
        ACTIVE = False
        times = []
        
        while joystick.connected:
            if(BENCHMARK):
                start_time = time.time()
                
            if(joystick['cross'] is not None):  # SQUARE EXIT
                print("EXIT")
                ser.close()
                break
                
            if(joystick['triangle'] is not None):  # TRIANGLE TOGGLE AGENT
                if(ACTIVE):
                    print("AGENT OFF")
                    ACTIVE = False   
                else:
                    print("AGENT ACTIVE")
                    ACTIVE = True
                time.sleep(0.2)
                
            image = (camera.value)
            img_gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
            img2 = cv2.resize(img_gray, (80,80))/255 # 60,40
            
            act_values = car_agent.model.predict(car_agent.conv_to_tensor(img2))
            agent_action = np.argmax(act_values[0])*10
            
            # Default
            THROTTLE = int((joystick['ly']+1)/2*180)
            STEERING = int((joystick['rx']+1)/2*180)
            
            if(ACTIVE):
                STEERING = agent_action
                THROTTLE = int(-160)
                
            # Manual 
            if(joystick['l1'] is not None):# L trigger for manual
                STEERING = int((joystick['rx']+1)/2*180)
                THROTTLE = int((joystick['ly']+1)/2*180)
                
            if(SHOW_LIVE):
                output = "{:05d}-{:05d}\n".format(int(THROTTLE), int(STEERING))
                ser.write(bytes(output,'utf-8'))
                speed = (ser.readline())
                if not speed:
                    speed = 0
                    
                info_str = "SPEED {:1d} PREDICT {:1d}".format(int(speed), int(agent_action))
            
                preview = img_gray
                cv2.putText(preview, info_str, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0))

                cv2.circle(preview, (int(main_cv.line[0]), int(122)), 4, (255,0,0),  -1)
                widget_test.value = bgr8_to_jpeg(preview)
                
                if(RECORD):
                    Frames.append(preview)
                    speeds.append(int(speed))

            if(BENCHMARK):
                delta = (time.time() - start_time)
                times.append(delta)
            
if(BENCHMARK):
    mean_delta = np.array(times).mean()
    fps = 1 / mean_delta
    print('average(sec):{:.2f}, fps:{:.2f}'.format(mean_delta, fps))

Image(value=b'', format='jpeg', height='122.0', width='244')

AGENT ACTIVE
AGENT OFF
EXIT
average(sec):0.01, fps:74.56


In [28]:
preview_widget = ipywidgets.Image(format='jpeg', width=width, height=height/2)
display(preview_widget)
print("SPEED AVG", np.mean(speeds))

out = cv2.VideoWriter("dqn.avi",cv2.VideoWriter_fourcc(*"MJPG"), 80,(width,height),0)
for f in Frames:
    out.write(f.astype('uint8'))
    preview_widget.value = bgr8_to_jpeg(f)
    
out.release()

Image(value=b'', format='jpeg', height='122.0', width='244')

SPEED AVG 259.2234161988773


In [ ]:
# RESTART CAM
camera.unobserve_all()
camera.stop()
time.sleep(2)
#camera = Camera(width=width, height=height, capture_width=1280, capture_height=720, capture_fps=60, capture_flip=2)

In [ ]:
camera.unobserve_all()
camera.stop()
car_agent.memory.clear()


In [17]:
car_agent.model.save("./model/DQN_BEST4.h5")